In [1]:
import pandas as pd             # data package
import matplotlib.pyplot as plt # graphics 
import datetime as dt
import numpy as np
from census import Census # This is new...

import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import os  

#import weightedcalcs as wc
#import numpy as np

import pyarrow as pa
import pyarrow.parquet as pq

In [2]:
#https://www.ons.gov.uk/economy/nationalaccounts/balanceofpayments/datasets/uktradecountrybycommodityexports
url = "https://www.ons.gov.uk/file?uri=%2feconomy%2fnationalaccounts%2fbalanceofpayments%2fdatasets%2fuktradecountrybycommodityexports%2fcurrent/cxcexportshistoricaldata.zip"

In [3]:
r = requests.get(url) 

brexit_FEB21 = zf.ZipFile(io.BytesIO(r.content))

brexit_FEB21.extractall(".\\data")

filename = brexit_FEB21.namelist()

In [4]:
df = pd.read_excel(".\\data" + "\\" + filename[0], sheet_name= "Country by Commodity Export")

In [5]:
df.head()

,COMMODITY,COUNTRY,DIRECTION,1997JAN,1997FEB,1997MAR,1997APR,1997MAY,1997JUN,1997JUL,...,2017MAR,2017APR,2017MAY,2017JUN,2017JUL,2017AUG,2017SEP,2017OCT,2017NOV,2017DEC
0,0 Food & live animals,AD Andorra,EX Exports,0.01,0.04,0.07,0.01,0.04,0.04,0.03,...,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.01,0.02
1,0 Food & live animals,AE United Arab Emirates,EX Exports,4.18,3.93,4.27,4.60,3.77,4.90,3.80,...,20.14,23.16,24.62,21.84,17.09,23.87,27.62,21.11,20.88,22.84
2,0 Food & live animals,AF Afghanistan,EX Exports,2.30,0.00,0.00,0.00,0.01,0.00,0.00,...,0.06,0.05,0.19,0.00,0.00,0.12,0.02,0.08,0.03,0.00
3,0 Food & live animals,AG Antigua and Barbuda,EX Exports,0.17,0.13,0.22,0.27,0.19,0.23,0.32,...,0.19,0.27,0.24,0.09,0.12,0.19,0.18,0.23,0.50,0.38
4,0 Food & live animals,AI Anguilla,EX Exports,0.00,0.00,0.01,0.03,0.00,0.00,0.00,...,0.00,0.01,0.00,0.00,0.05,0.00,0.00,0.00,0.04,0.00


In [6]:
df = df.melt(id_vars = ["COMMODITY", "COUNTRY", "DIRECTION"])

df.drop("DIRECTION", axis = 1, inplace = True)

df.variable= pd.to_datetime(df.variable, format="%Y%b")

df.rename({"variable":"time", "value": "exports"},inplace = True, axis = 1)

df.COUNTRY = df.COUNTRY.str[3:]

df.exports = df.exports*1000000

In [7]:
grp = df[df.COMMODITY == 'T Total'].groupby("COUNTRY")

test = grp.agg({"exports": "sum"})

In [8]:
topcntry = test.sort_values(by = "exports", ascending = False).iloc[0:23].index.to_list()

In [9]:
df = df[df.COUNTRY.isin(topcntry)]

In [10]:
df.COMMODITY.replace({"T Total": "Total"}, inplace = True)
df.COUNTRY.replace({"United States inc Puerto Rico": "United States"}, inplace = True)
df.COUNTRY.replace({"Extra EU 28 (Rest of World)": "Excl. EU 28 (Rest of World)"}, inplace = True)

In [11]:
df

,COMMODITY,COUNTRY,time,exports
1,0 Food & live animals,United Arab Emirates,1997-01-01,4.180000e+06
12,0 Food & live animals,Australia,1997-01-01,4.950000e+06
15,0 Food & live animals,Total EU(28),1997-01-01,3.850000e+08
19,0 Food & live animals,Belgium,1997-01-01,2.255000e+07
36,0 Food & live animals,Canada,1997-01-01,8.180000e+06
...,...,...,...,...
7465447,Total,Sweden,2017-12-01,4.100000e+08
7465448,Total,Singapore,2017-12-01,4.940000e+08
7465474,Total,Turkey,2017-12-01,4.940000e+08
7465482,Total,United States,2017-12-01,3.969000e+09


In [12]:
grp = df.groupby(["COUNTRY"])

In [13]:
def ex_metal_total(df):
        
    if not (df[df["COMMODITY"] == "9 Unspecified goods"].empty):
                
        foo = pd.DataFrame([])

        fooall = df[df["COMMODITY"] == "Total"].set_index(["time"])["exports"] 
        
        foobar = df[df["COMMODITY"] == "9 Unspecified goods"].set_index(["time"])["exports"]

        foo["exports"] = fooall - foobar
    # Grab the total and then sbutract off the unspecified goods
    
        foo["COUNTRY"] = df.COUNTRY.unique()[0]
    
        #print(df.COUNTRY.unique())
    
        foo["COMMODITY"] = "Total (ex. metals)"
        
        foo.reset_index(inplace = True)
        
        foo[df.columns]
        
        df = df.append(foo)
    
    return df

In [14]:
df = grp.apply(ex_metal_total)

In [15]:
df.drop(["COUNTRY"], axis = 1, inplace = True)

df.reset_index(inplace = True)

df.drop(["level_1"], axis = 1, inplace = True)

In [16]:
df.set_index(["COUNTRY", "COMMODITY", "time"], inplace = True)

In [17]:
df.head()

exports
COUNTRY   COMMODITY                   time                 
Australia 0 Food & live animals       1997-01-01  4950000.0
          00 Live animals             1997-01-01  1130000.0
          01 Meat & meat preparations 1997-01-01        0.0
          02 Dairy products & eggs    1997-01-01   140000.0
          03 Fish & shellfish         1997-01-01   360000.0

In [19]:
out_file = ".\\data"+ "\\UK-exports-1997-2017.parquet"

pq.write_table(pa.Table.from_pandas(df.sort_index()), out_file)

In [20]:
file = ".\\data"+ "\\UK-exports-1997-2017.parquet"

foo = pq.read_table(file).to_pandas()

In [21]:
df.tail()

exports
COUNTRY     COMMODITY          time                    
Whole world Total (ex. metals) 2017-08-01  2.597800e+10
                               2017-09-01  2.849800e+10
                               2017-10-01  2.978700e+10
                               2017-11-01  3.006700e+10
                               2017-12-01  2.683200e+10